# Recommendation Systems

Recommendation systems have revolutionized the way we discover and explore new things. These intelligent systems utilize sophisticated algorithms and data analysis to understand individual preferences and provide personalized recommendations. 
<br><br>
By analyzing user data, such as **browsing history**, **purchase patterns**, and **social interactions**, recommendation systems can effectively predict and suggest items that align with users' interests. Whether it's suggesting a new movie to watch, a book to read, or a product to buy, these systems streamline decision-making and enhance the overall user experience. 
<br>
With their ability to uncover hidden gems and introduce users to exciting possibilities, recommendation systems have become invaluable tools in navigating the overwhelming abundance of choices in today's digital landscape.

### Recommendation Systems and Vector Databases

In recommendation systems, understanding the similarity between users and items is crucial for generating accurate and personalized recommendations. By leveraging vector databases, these systems can store and organize user and item vectors, which capture the essential characteristics and preferences associated with each user and item. 
<br><br>
The vector database employs <a href="https://www.pinecone.io/learn/vector-database/#:~:text=a%20vector%20database.-,Algorithms,-Several%20algorithms%20can">advanced indexing techniques</a>, to enable fast retrieval of **similar users or items** based on their **vector representations**. This enables recommendation systems to efficiently process *large-scale datasets* and identify meaningful connections, leading to more precise and relevant recommendations. 
<br><br>
By harnessing the power of vector databases, such as **Pinecone**, recommendation systems can optimize their performance, enhance user satisfaction, and deliver tailored experiences that align with individual preferences.
<br><br><br>
Let's take a look at how we can implement one of those use cases!

We start by installing all necessary libraries.

In [1]:
!pip install -qU \
    "pinecone-client[grpc]"==2.2.1 \
    pinecone-datasets \
    transformers \
    tensorflow

You should consider upgrading via the '/home/jelena/.pyenv/versions/3.8.16/envs/onboarding-recommender-pinecone/bin/python3.8 -m pip install --upgrade pip' command.


---

🚨 _Note: the above `pip install` is formatted for Jupyter notebooks. If running elsewhere you may need to drop the `!`._

---

## Data Preparation

<img alt="Onboarding recommender diagram" src="/home/jelena/Projects/Pinecone/examples/recommendation/onboarding-recommender/assets/onboarding_recommender_data_flow.jpg"  width="70%">

#### Downloading the Dataset

We will download a pre-embedding dataset from `pinecone-datasets`. Allowing us to skip the embedding and any other preprocessing steps. 
<br><br>
When working with your own dataset you will need to perform this embedding step but we have prebuilt the embeddings so we can jump right to the action.

In [2]:
DATASET_NAME = "movielens-user-ratings"

In [3]:
from pinecone_datasets import load_dataset

dataset = load_dataset(DATASET_NAME)
dataset.head()

,id,values,sparse_values,metadata,blob
0,tt5027774,"[-0.12388430535793304, 0.23021861910820007, -0...",None,None,"{'imdb_id': 'tt5027774', 'movie_id': 6705, 'po..."
1,tt5463162,"[0.008479624055325985, 0.3665461540222168, -0....",None,None,"{'imdb_id': 'tt5463162', 'movie_id': 7966, 'po..."
2,tt4007502,"[-0.0022702165879309177, 0.5886886715888977, -...",None,None,"{'imdb_id': 'tt4007502', 'movie_id': 1614, 'po..."
3,tt4209788,"[0.08350061625242233, 0.4322584867477417, -0.2...",None,None,"{'imdb_id': 'tt4209788', 'movie_id': 7022, 'po..."
4,tt2948356,"[-0.1614755392074585, 0.41389355063438416, -0....",None,None,"{'imdb_id': 'tt2948356', 'movie_id': 3571, 'po..."


In [4]:
len(dataset)

970582

#### Reformatting the Dataset

A `pinecone-dataset` always contains `id`, `values`, `sparse_values`, `metadata`, and `blob`. All we need are the IDs, vector embeddings (stored in `values`), and some metadata (which is actually stored in `blob`). Let's reformat the dataset ready for adding to Pinecone. We also drop `sparse_values` as they are not needed for this example.


In [5]:
dataset.documents.drop(['sparse_values', 'metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)

dataset.head()

,id,values,metadata
0,tt5027774,"[-0.12388430535793304, 0.23021861910820007, -0...","{'imdb_id': 'tt5027774', 'movie_id': 6705, 'po..."
1,tt5463162,"[0.008479624055325985, 0.3665461540222168, -0....","{'imdb_id': 'tt5463162', 'movie_id': 7966, 'po..."
2,tt4007502,"[-0.0022702165879309177, 0.5886886715888977, -...","{'imdb_id': 'tt4007502', 'movie_id': 1614, 'po..."
3,tt4209788,"[0.08350061625242233, 0.4322584867477417, -0.2...","{'imdb_id': 'tt4209788', 'movie_id': 7022, 'po..."
4,tt2948356,"[-0.1614755392074585, 0.41389355063438416, -0....","{'imdb_id': 'tt2948356', 'movie_id': 3571, 'po..."


Here is an example of the metadata value.

In [217]:
from pprint import pp

pp(dataset.documents['metadata'][0])

{'imdb_id': 'tt5027774',
 'movie_id': 6705,
 'poster': 'https://m.media-amazon.com/images/M/MV5BMjI0ODcxNzM1N15BMl5BanBnXkFtZTgwMzIwMTEwNDI@._V1_SX300.jpg',
 'rating': 4.0,
 'title': 'Three Billboards Outside Ebbing, Missouri (2017)',
 'user_id': 4556}


Now we move on to initializing our Pinecone vector database.

## Creating an Index

In [7]:
import os
import pinecone
import time

/home/jelena/.pyenv/versions/3.8.16/envs/onboarding-recommender-pinecone/lib/python3.8/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


We set `PINECONE_API_KEY` and `PINECONE_ENV` variables that we are going to use during initialization step. You can find these values in [Pinecone Console](https://app.pinecone.io/) in the API Keys section.

In [8]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
PINECONE_ENV = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

Now we can use these variables for initialization.

In [9]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

In order to create a new index, we need to specify the index name, similarity metric, as well as the dimension of the vectors stored in that index. 
<br>
We will assign these values here. 
<br>
Note that the dimension parameter has to match the embedding dimensions provided in the dataset (or the model that outputs those embeddings).

In [10]:
# embedding dimensions
len(dataset.documents['values'][0])

32

In [11]:
INDEX_NAME = 'onboarding-recommender'
SIM_METRIC = 'cosine'
DIMENSION = 32

First, we need to check if the index already exists. In this example, we will delete it and create a new one.

In [12]:
if INDEX_NAME in pinecone.list_indexes():
    pinecone.delete_index(INDEX_NAME)

pinecone.create_index(
    name=INDEX_NAME,
    metric=SIM_METRIC,
    dimension=DIMENSION,
)
# wait a moment for the index to be fully initialized
time.sleep(1)

We are going to initialize an index variable so that we can use it later on to describe the index and perform vector upsert.

In [14]:
grpc_index = pinecone.GRPCIndex(INDEX_NAME)

In [15]:
grpc_index.describe_index_stats()

{'dimension': 32,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [16]:
grpc_index.upsert_from_dataframe(dataset.documents)

sending upsert requests: 100%|██████████| 970582/970582 [01:02<00:00, 15522.52it/s]


upserted_count: 970582

In [81]:
grpc_index.describe_index_stats()

{'dimension': 32,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10269}},
 'total_vector_count': 10269}

## Querying the Index

Now, when the index is populated, we can perform queries on it to find the most relevant recommendations.
<br>
To do that, we need to instantiate our embedding models so that we can create vectors from our input user or input item objects.

### Getting the Model

We will download the models from the HuggingFace Hub. We will use one model to embed the *example user* and another model to embed the *example item*. <br>
This will allow us to retrieve the most relevant items for a specific user or find the most similar items to a specific item.

In [17]:
from huggingface_hub import from_pretrained_keras

user_model = from_pretrained_keras("pinecone/movie-recommender-user-model")
movie_model = from_pretrained_keras("pinecone/movie-recommender-movie-model")

2023-06-13 10:42:05.535914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 10:42:08.080937: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-13 10:42:08.080959: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-13 10:42:11.553871: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

config.json not found in HuggingFace Hub.
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 1696.33it/s]


Before we proceed, we can create a `movie_details` dataset that we can use later on to print out the results.

In [218]:
import pandas as pd

movies_details = pd.DataFrame(dataset.documents['metadata'].values.tolist())
movies_details.head()

,imdb_id,movie_id,poster,rating,title,user_id
0,tt5027774,6705,https://m.media-amazon.com/images/M/MV5BMjI0OD...,4.0,"Three Billboards Outside Ebbing, Missouri (2017)",4556
1,tt5463162,7966,https://m.media-amazon.com/images/M/MV5BMDkzNm...,3.5,Deadpool 2 (2018),20798
2,tt4007502,1614,https://m.media-amazon.com/images/M/MV5BMjY3YT...,4.5,Frozen Fever (2015),26543
3,tt4209788,7022,https://m.media-amazon.com/images/M/MV5BNTkzMz...,4.0,Molly's Game (2017),4106
4,tt2948356,3571,https://m.media-amazon.com/images/M/MV5BOTMyMj...,4.0,Zootopia (2016),15259


#### Item Similarity

First, we can check how our vector database behaves when returning the most similar movies upon querying it using the movie vector created using the `movie_model` loaded above.

In [223]:
movie_id = 1263  # you can try experimenting with different movie ids to obtain different results, for example 3571
movie_vector = movie_model(movie_id).numpy().tolist()

In [224]:
movies_details[movies_details['movie_id'] == movie_id]['title'].tolist()[0]

'Avengers: Infinity War - Part I (2018)'

In [225]:
movie_query_results = grpc_index.query(queries=[movie_vector], top_k=10, include_metadata=True)

In [227]:
for res in movie_query_results.results:
    df = pd.DataFrame(
        {
            'movies': [match.metadata['title'] for match in res.matches],
            'scores': [match.score for match in res.matches]
        }
    )
    print("Recommendations: ")
    display(df)

Recommendations: 


,movies,scores
0,Avengers: Infinity War - Part I (2018),1.000000
1,Avengers: Infinity War - Part II (2019),0.987279
2,Thor: Ragnarok (2017),0.981357
3,Captain America: Civil War (2016),0.978873
4,Guardians of the Galaxy (2014),0.976149
5,Guardians of the Galaxy 2 (2017),0.960252
6,Avengers: Age of Ultron (2015),0.945295
7,Untitled Spider-Man Reboot (2017),0.943886
8,Logan (2017),0.936165
9,Star Wars: Dresca,0.933541


We can observe that it is doing an excellent job in finding similar movies, and it is accomplishing this task very quickly.

#### User Recommendations

Now, let's observe how our vector database behaves when we query it using the user vector.
<br>
We expect to receive movies that closely resemble the ones that the user rated highly.

In [212]:
user_id = 3
user_vector = user_model(user_id).numpy().tolist()

Here, we are defining a function that allows us to easily display the movies that the user rated in the past.

In [213]:
def top_movies_user_rated(user):
    # get list of movies that the user has rated
    user_movies = movies_details[movies_details["user_id"] == user]
    # order by their top rated movies
    top_rated = user_movies.sort_values(by=['rating'], ascending=False)
    # return the top 14 movies
    return pd.DataFrame(
        {
            'movies': top_rated['title'].tolist()[:14],
            'ratings': top_rated['rating'].tolist()[:14]
        }
    )

In [215]:
display(top_movies_user_rated(user_id))

,movies,ratings
0,Big Hero 6 (2014),4.5
1,Captain America: Civil War (2016),4.0
2,Avengers: Age of Ultron (2015),4.0
3,Arrival (2016),2.5
4,The Martian (2015),2.5


And now we can pass our `user_vector` to the query to get the recommendations.

In [214]:
query_results = grpc_index.query(queries=[user_vector], top_k=10, include_metadata=True)

In [216]:
for res in query_results.results:
    df = pd.DataFrame(
        {
            'movies': [match.metadata['title'] for match in res.matches],
            'scores': [match.score for match in res.matches]
        }
    )
    print("Recommendations: ")
    display(df)

Recommendations: 


,movies,scores
0,Big Hero 6 (2014),0.854731
1,Captain America: Civil War (2016),0.849430
2,Avengers: Age of Ultron (2015),0.834037
3,The Witch Files (2018),0.830561
4,Avengers: Infinity War - Part I (2018),0.829733
5,Untitled Spider-Man Reboot (2017),0.825950
6,Monster High: 13 Wishes (2013),0.824344
7,Guardians of the Galaxy 2 (2017),0.822629
8,Lovestruck: The Musical (2013),0.820314
9,Spider-Man: Far from Home (2019),0.820075


Again, we can observe that these recommendations strongly resemble the movies that the user rated highly, and there are no movies similar to the ones that the user rated with a low value.

## Summary

The notebook demonstrated the step-by-step process of creating and populating an index in the vector database. It covered aspects such as specifying the index name, similarity metric, and vector dimensions. The example also included instructions on checking if an index exists, deleting and creating new indexes when necessary.

Furthermore, the notebook illustrated the usage of embedding models to generate vector representations of both users and items. The results showed that the recommendations closely resembled the movies that the user rated highly, and dissimilar movies were not included.

Overall, this example showcased the power and efficiency of vector databases in recommendation systems. It is important to note that the benefits of vector databases extend beyond movies and can be applied to various types of items, making them a valuable tool in building effective recommendation systems.